In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext tensorboard

In [3]:
import numpy as np
import tensorflow as tf

from domoku.tools import GomokuTools as Gt

In [4]:
input_size=7

In [5]:
import os
import tempfile
from pathlib import Path

tempdir = tempfile.mkdtemp()

data_dir = Path(os.environ.get('MODELS_DIR', tempdir)) / 'gomoku'
models_dir = data_dir / 'models'
logs_dir = data_dir / 'logs'
cp_dir = data_dir / 'checkpoints'
print(f"Checkpoints can be found in {cp_dir}")
print(f"Models can be found in {models_dir}")
print(f"Logs can be found in {logs_dir}")

Checkpoints can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpg3gccj8a/gomoku/checkpoints
Models can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpg3gccj8a/gomoku/models
Logs can be found in /var/folders/0s/fny92xb54c50mb90g3l_zmx00000gp/T/tmpg3gccj8a/gomoku/logs


---

# Tutorial: Training a Conv Model
#### Detecting Lines of Three on a Board of 7x7

## Sampling Random Boards

In [6]:
from notebooks.ml_basics_recap.data import new_sample
sample = new_sample(board_size=input_size, num_blacks=20, num_whites=0)
sample.shape

(7, 7, 2)

In [7]:
Gt.print_bin(sample, True)

shape: (7, 7, 2)
[[1. 0. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1.]]


---

## The Detection Map

In [8]:
from notebooks.ml_basics_recap.models.heuristic_detector import HeuristicDetector
detector = HeuristicDetector(input_size)

2022-06-11 13:18:46.447456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Notice the additional dimension for the batch size come and go

In [9]:
tf.floor(detector(np.expand_dims(sample, 0))+.01)

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>

In [10]:
np.squeeze(detector(np.expand_dims(sample, 0))).shape

(7, 7)

---

## Batches From a Dataset
#### Strong Advice: Meticulously observe the shape of all incoming data!

In [11]:
BATCH_SIZE=4

In [12]:
from notebooks.ml_basics_recap.data import new_sample, new_dataset
dataset = new_dataset(100, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)

In [13]:
iterator  = iter(dataset)
states, labels = iterator.next()
print (f"Inputs' shape: {states.shape}, Labels' shape: {labels.shape}")

Inputs' shape: (4, 7, 7, 2), Labels' shape: (4, 7, 7)


---

## The Trainable Model

In [14]:
from notebooks.ml_basics_recap.models import SimpleConvQFunction

model_q = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)
print(model_q(states).shape)
print(np.squeeze(model_q(np.expand_dims(states, 0))).shape)

(4, 5, 5, 1)
(4, 5, 5)


---

## Training

In [15]:
TRAIN_SIZE = 1024 * 8
TEST_SIZE = 1024
BATCH_SIZE = 256

### Create a dataset from the heuristics

In [16]:
train_dataset = new_dataset(size=TRAIN_SIZE,
                            sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                            labeler=detector, separate=False).batch(BATCH_SIZE)

test_dataset = new_dataset(size=TEST_SIZE,
                           sampler=lambda: new_sample(board_size=input_size, num_blacks=20, num_whites=0),
                           labeler=detector, separate=False).batch(BATCH_SIZE)

In [17]:
import tensorflow as tf

In [18]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

In [19]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.MeanSquaredError('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.MeanSquaredError('test_accuracy')

In [20]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [24]:
import datetime as dt
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [25]:
iterator  = iter(train_dataset)
x_train, y_train = iterator.next()

In [26]:
flatten = tf.keras.layers.Flatten()
model = SimpleConvQFunction(input_size, n_filters=8, n_layers=4)
labels = model(x_train, training=True)
loss = loss_object(y_train, labels)
model.summary()

Model: "simple_conv_q_function_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          multiple                  152       
                                                                 
 conv2d_13 (Conv2D)          multiple                  584       
                                                                 
 conv2d_14 (Conv2D)          multiple                  584       
                                                                 
 conv2d_15 (Conv2D)          multiple                  584       
                                                                 
 conv2d_16 (Conv2D)          multiple                  73        
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


In [27]:
EPOCHS = 40

for epoch in range(EPOCHS):
    for x_train, y_train in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for x_test, y_test in test_dataset:
        test_step(model, x_test, y_test)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)


    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.06899061053991318, Accuracy: 6.89906120300293, Test Loss: 0.06689083576202393, Test Accuracy: 6.689083576202393
Epoch 2, Loss: 0.06471351534128189, Accuracy: 6.4713521003723145, Test Loss: 0.058587975800037384, Test Accuracy: 5.858797550201416
Epoch 3, Loss: 0.05177763104438782, Accuracy: 5.17776346206665, Test Loss: 0.046572066843509674, Test Accuracy: 4.6572065353393555
Epoch 4, Loss: 0.04341210797429085, Accuracy: 4.341209888458252, Test Loss: 0.040649451315402985, Test Accuracy: 4.064945220947266
Epoch 5, Loss: 0.037916310131549835, Accuracy: 3.7916297912597656, Test Loss: 0.03587673231959343, Test Accuracy: 3.5876731872558594
Epoch 6, Loss: 0.03318950906395912, Accuracy: 3.318951368331909, Test Loss: 0.03117094188928604, Test Accuracy: 3.1170942783355713
Epoch 7, Loss: 0.02914777770638466, Accuracy: 2.914778709411621, Test Loss: 0.027891524136066437, Test Accuracy: 2.7891526222229004
Epoch 8, Loss: 0.026177197694778442, Accuracy: 2.6177191734313965, Test Loss: 0.0

In [28]:
BATCH_SIZE=1

In [29]:
dataset = new_dataset(10, lambda: new_sample(input_size, 20, 0), detector).batch(BATCH_SIZE)
iterator  = iter(dataset)

### Labels vs Learned

In [31]:
states, labels = iterator.next()
pred = np.squeeze(model(states))

Gt.print_bin(states, combine=True)
print(tf.floor(labels+.01))
print(np.round(pred, 2))

shape: (7, 7, 2)
[[0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 0. 1. 0. 1. 1.]
 [1. 1. 0. 0. 1. 1. 0.]]
tf.Tensor(
[[[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 1. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 7, 7), dtype=float32)
[[ 0.   -0.   -0.   -0.01 -0.   -0.   -0.  ]
 [-0.   -0.    0.   -0.   -0.   -0.   -0.  ]
 [-0.    0.    0.    0.   -0.    0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.01  0.    0.  ]
 [ 0.01  0.   -0.01 -0.    0.01  0.01  0.01]
 [-0.01  1.    0.01  0.99  0.    0.99 -0.01]
 [-0.02 -0.01  0.01  0.01 -0.   -0.02  0.02]]


In [32]:
train_log_dir

'logs/gradient_tape/20220611-132056/train'

In [34]:
%tensorboard --logdir logs/gradient_tape/20220611-132056/train

Launching TensorBoard...